In [ ]:
# ============================================================================
# CELDA 1: INSTALACIÓN DE LIBRERÍAS NECESARIAS
# ============================================================================
import sys
import subprocess

print("="*70)
print("VERIFICANDO LIBRERÍAS NECESARIAS")
print("="*70)
print()

librerias = {
    'pandas': 'Manipulación y análisis de datos',
    'numpy': 'Operaciones numéricas',
    'openpyxl': 'Leer/escribir archivos Excel',
    'matplotlib': 'Gráficos y visualizaciones',
}

for libreria, descripcion in librerias.items():
    try:
        __import__(libreria)
        print(f"   ✅ {libreria:.<20} {descripcion} - YA INSTALADA")
    except ImportError:
        print(f"   ❌ {libreria:.<20} {descripcion} - FALTANTE")
        try:
            print(f"      Instalando {libreria}...")
            subprocess.check_call(
                [sys.executable, "-m", "pip", "install", libreria],
                stdout=subprocess.DEVNULL,
                stderr=subprocess.DEVNULL
            )
            print(f"      ✅ {libreria} instalada correctamente")
        except:
            print(f"      ⚠️  Error instalando {libreria}")

print()
print("="*70)
print("VERIFICANDO VERSIONES")
print("="*70)

import pandas as pd
import numpy as np
import matplotlib

print(f"\n✅ Python:      {sys.version.split()[0]}")
print(f"✅ Pandas:      {pd.__version__}")
print(f"✅ NumPy:       {np.__version__}")
print(f"✅ Matplotlib:  {matplotlib.__version__}")

print()
print("="*70)
print("✅ SISTEMA LISTO")
print("="*70)
print()

In [ ]:
# ============================================================================
# CELDA 2: CONFIGURACIÓN DEL ANÁLISIS POR PRODUCTOR
# ============================================================================

import warnings
warnings.filterwarnings('ignore')

# Configuración de pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 100)

# Configuración de matplotlib
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (16, 8)
plt.rcParams['font.size'] = 11

print("="*70)
print("PARÁMETROS DEL SEMÁFORO POR PRODUCTOR")
print("="*70)
print()

# ============================================================================
# PARÁMETROS EDITABLES
# ============================================================================

# 1. FECHA DE INICIO DE COSECHA (Día-Mes-Año)
FECHA_INICIO_COSECHA = '03-04-2025'

# 2. MARGEN DE STOCKEO (días permitidos para acumular bins)
MARGEN_STOCKEO = 90  # días

# 3. DÍAS SIN DEVOLUCIÓN (para considerar inactivo)
DIAS_SIN_DEVOLUCION = 14  # días

# 4. DEUDA RELATIVA MÁXIMA (porcentaje del total pedido)
DEUDA_RELATIVA_MAXIMA = 40  # porcentaje

# ============================================================================
# UMBRALES DEL SEMÁFORO (basados en deuda relativa)
# ============================================================================

# Verde: Situación óptima
UMBRAL_VERDE = 20  # Menos del 20% de deuda

# Amarillo: Requiere monitoreo
UMBRAL_AMARILLO_MIN = 20
UMBRAL_AMARILLO_MAX = 40

# Rojo: Requiere acción
UMBRAL_ROJO = 40  # Más del 40% de deuda

# ============================================================================
# ARCHIVO DE DATOS
# ============================================================================

RUTA_MOVIMIENTOS = r"C:\JUGOS\resultados\MOVIMIENTOS.xlsx"

print(f"📅 Fecha inicio cosecha:        {FECHA_INICIO_COSECHA}")
print(f"⏱️  Margen de stockeo:           {MARGEN_STOCKEO} días")
print(f"⏳ Días sin devolución:         {DIAS_SIN_DEVOLUCION} días")
print(f"📊 Deuda relativa máxima:       {DEUDA_RELATIVA_MAXIMA}%")
print()
print(f"🟢 Verde (deuda < {UMBRAL_VERDE}%)")
print(f"🟡 Amarillo (deuda {UMBRAL_AMARILLO_MIN}% - {UMBRAL_AMARILLO_MAX}%)")
print(f"🔴 Rojo (deuda > {UMBRAL_ROJO}%)")
print()
print("="*70)
print("✅ CONFIGURACIÓN COMPLETADA")
print("="*70)
print()

In [ ]:
# ============================================================================
# CELDA 3: CARGAR ARCHIVO MOVIMIENTOS.xlsx
# ============================================================================

print("="*70)
print("CARGANDO ARCHIVO: MOVIMIENTOS.xlsx")
print("="*70)
print()

try:
    # Cargar archivo
    MOVIMIENTOS = pd.read_excel(RUTA_MOVIMIENTOS)
    
    print(f"✅ Archivo cargado exitosamente")
    print()
    print(f"📊 Total de registros: {len(MOVIMIENTOS):,}")
    print(f"📋 Columnas: {', '.join(MOVIMIENTOS.columns)}")
    print()
    
    # Validar columnas requeridas
    columnas_requeridas = ['FECHA', 'PROVEEDOR', 'NOMBRE', 'CANTIDAD', 'MOVIMIENTO']
    columnas_faltantes = [col for col in columnas_requeridas if col not in MOVIMIENTOS.columns]
    
    if columnas_faltantes:
        print(f"⚠️  COLUMNAS FALTANTES: {', '.join(columnas_faltantes)}")
        raise ValueError("Faltan columnas requeridas")
    
    print("✅ Todas las columnas requeridas están presentes")
    print()
    
    # Convertir fechas
    MOVIMIENTOS['FECHA_DIA'] = pd.to_datetime(MOVIMIENTOS['FECHA']).dt.date
    MOVIMIENTOS['FECHA_DIA'] = pd.to_datetime(MOVIMIENTOS['FECHA_DIA'])
    
    # Convertir fecha de inicio de cosecha
    FECHA_COSECHA = pd.to_datetime(FECHA_INICIO_COSECHA, format='%d-%m-%Y')
    
    print("="*70)
    print("INFORMACIÓN GENERAL")
    print("="*70)
    print()
    print(f"📅 Rango de fechas:")
    print(f"   Desde: {MOVIMIENTOS['FECHA_DIA'].min().strftime('%d/%m/%Y')}")
    print(f"   Hasta: {MOVIMIENTOS['FECHA_DIA'].max().strftime('%d/%m/%Y')}")
    print()
    print(f"👥 Total productores: {MOVIMIENTOS['PROVEEDOR'].nunique():,}")
    print()
    print(f"📊 Movimientos por tipo:")
    print(MOVIMIENTOS['MOVIMIENTO'].value_counts())
    print()
    
    print("="*70)
    print("✅ DATOS CARGADOS CORRECTAMENTE")
    print("="*70)
    print()
    
except FileNotFoundError:
    print(f"❌ ERROR: No se encontró el archivo en la ruta: {RUTA_MOVIMIENTOS}")
    print()
    print("📝 Verifica:")
    print("   1. Que la ruta del archivo sea correcta")
    print("   2. Que el archivo MOVIMIENTOS.xlsx existe en esa ubicación")
    print("   3. Que ejecutaste el NOTEBOOK 1 (Preparación) primero")
    print()
    raise
except Exception as e:
    print(f"❌ ERROR: {str(e)}")
    raise

In [ ]:
# ============================================================================
# CELDA 4: CALCULAR MÉTRICAS POR PRODUCTOR
# ============================================================================

print("="*70)
print("ANÁLISIS POR PRODUCTOR - CÁLCULO DE MÉTRICAS")
print("="*70)
print()

# Fecha de hoy (última fecha en los datos)
FECHA_HOY = MOVIMIENTOS['FECHA_DIA'].max()

print(f"📅 Fecha de análisis: {FECHA_HOY.strftime('%d/%m/%Y')}")
print(f"📅 Fecha inicio cosecha: {FECHA_COSECHA.strftime('%d/%m/%Y')}")
print()

# Crear DataFrame para resultados
resultados = []

# Obtener lista de productores únicos
productores = MOVIMIENTOS['PROVEEDOR'].unique()

print(f"👥 Analizando {len(productores):,} productores...")
print()

for i, proveedor in enumerate(productores, 1):
    # Filtrar movimientos del productor
    df_prod = MOVIMIENTOS[MOVIMIENTOS['PROVEEDOR'] == proveedor].copy()
    
    # Obtener nombre
    nombre = df_prod['NOMBRE'].iloc[0]
    
    # Calcular totales
    salidas_total = df_prod[df_prod['MOVIMIENTO'] == 'SALIDA']['CANTIDAD'].sum()
    entradas_total = df_prod[df_prod['MOVIMIENTO'] == 'ENTRADA']['CANTIDAD'].sum()
    deuda_actual = salidas_total - entradas_total
    
    # Calcular deuda relativa (porcentaje)
    if salidas_total > 0:
        deuda_relativa = (deuda_actual / salidas_total) * 100
    else:
        deuda_relativa = 0
    
    # Fecha del primer pedido (primera salida)
    df_salidas = df_prod[df_prod['MOVIMIENTO'] == 'SALIDA']
    if len(df_salidas) > 0:
        fecha_primer_pedido = df_salidas['FECHA_DIA'].min()
        dias_desde_primer_pedido = (FECHA_HOY - fecha_primer_pedido).days
    else:
        fecha_primer_pedido = None
        dias_desde_primer_pedido = 0
    
    # Última devolución (última entrada)
    df_entradas = df_prod[df_prod['MOVIMIENTO'] == 'ENTRADA']
    if len(df_entradas) > 0:
        fecha_ultima_entrada = df_entradas['FECHA_DIA'].max()
        dias_sin_devolucion = (FECHA_HOY - fecha_ultima_entrada).days
    else:
        fecha_ultima_entrada = None
        dias_sin_devolucion = 999  # Nunca devolvió
    
    # Determinar si está dentro del margen de stockeo
    dentro_margen = dias_desde_primer_pedido <= MARGEN_STOCKEO
    
    # Calcular ratio de devolución
    if salidas_total > 0:
        ratio_devolucion = (entradas_total / salidas_total) * 100
    else:
        ratio_devolucion = 0
    
    # Guardar resultados
    resultados.append({
        'PROVEEDOR': proveedor,
        'NOMBRE': nombre,
        'SALIDAS_TOTAL': salidas_total,
        'ENTRADAS_TOTAL': entradas_total,
        'DEUDA_BINS': deuda_actual,
        'DEUDA_RELATIVA_%': round(deuda_relativa, 1),
        'RATIO_DEVOLUCION_%': round(ratio_devolucion, 1),
        'FECHA_PRIMER_PEDIDO': fecha_primer_pedido,
        'DIAS_DESDE_PRIMER_PEDIDO': dias_desde_primer_pedido,
        'FECHA_ULTIMA_ENTRADA': fecha_ultima_entrada,
        'DIAS_SIN_DEVOLUCION': dias_sin_devolucion,
        'DENTRO_MARGEN': dentro_margen
    })

# Crear DataFrame de resultados
df_productores = pd.DataFrame(resultados)

print(f"✅ Métricas calculadas para {len(df_productores):,} productores")
print()
print("="*70)
print("✅ CELDA 4 COMPLETADA")
print("="*70)

In [ ]:
# ============================================================================
# CELDA 5: APLICAR LÓGICA DEL SEMÁFORO
# ============================================================================

print("="*70)
print("APLICANDO LÓGICA DEL SEMÁFORO")
print("="*70)
print()

def determinar_semaforo(row):
    """
    Determina el color del semáforo para un productor
    """
    deuda_relativa = row['DEUDA_RELATIVA_%']
    dias_sin_devolucion = row['DIAS_SIN_DEVOLUCION']
    dentro_margen = row['DENTRO_MARGEN']
    deuda_bins = row['DEUDA_BINS']
    
    # CASO 1: Dentro del margen de stockeo
    if dentro_margen:
        return 'VERDE', 'Dentro del margen de stockeo'
    
    # CASO 2: Deuda muy baja (siempre verde)
    if deuda_relativa < UMBRAL_VERDE:
        return 'VERDE', f'Deuda baja ({deuda_relativa:.1f}%)'
    
    # CASO 3: Rojo - Sin actividad reciente + deuda alta
    if dias_sin_devolucion > DIAS_SIN_DEVOLUCION and deuda_relativa > UMBRAL_ROJO:
        return 'ROJO', f'{dias_sin_devolucion} días sin devolver + deuda {deuda_relativa:.1f}%'
    
    # CASO 4: Rojo - Deuda muy alta (sin importar actividad)
    if deuda_relativa > 60:
        return 'ROJO', f'Deuda crítica ({deuda_relativa:.1f}%)'
    
    # CASO 5: Verde - Está devolviendo activamente
    if dias_sin_devolucion <= 7 and deuda_relativa <= UMBRAL_AMARILLO_MAX:
        return 'VERDE', f'Activo (última devolución hace {dias_sin_devolucion} días)'
    
    # CASO 6: Amarillo - Situación intermedia
    if deuda_relativa <= UMBRAL_AMARILLO_MAX:
        return 'AMARILLO', f'{dias_sin_devolucion} días sin devolver, deuda {deuda_relativa:.1f}%'
    
    # CASO 7: Rojo - Por defecto si supera umbrales
    return 'ROJO', f'Deuda alta ({deuda_relativa:.1f}%) + {dias_sin_devolucion} días inactivo'

# Aplicar la lógica a cada productor
df_productores[['SEMAFORO', 'RAZON']] = df_productores.apply(
    lambda row: pd.Series(determinar_semaforo(row)), axis=1
)

print("✅ Semáforo aplicado a todos los productores")
print()

# Contar por color
conteo = df_productores['SEMAFORO'].value_counts()

print("="*70)
print("RESUMEN POR COLOR")
print("="*70)
print()
print(f"🟢 VERDE:    {conteo.get('VERDE', 0):>5} productores ({conteo.get('VERDE', 0)/len(df_productores)*100:>5.1f}%)")
print(f"🟡 AMARILLO: {conteo.get('AMARILLO', 0):>5} productores ({conteo.get('AMARILLO', 0)/len(df_productores)*100:>5.1f}%)")
print(f"🔴 ROJO:     {conteo.get('ROJO', 0):>5} productores ({conteo.get('ROJO', 0)/len(df_productores)*100:>5.1f}%)")
print()

print("="*70)
print("✅ CELDA 5 COMPLETADA")
print("="*70)

In [ ]:
# ============================================================================
# CELDA 6: PRODUCTORES EN ROJO - REQUIEREN ATENCIÓN INMEDIATA
# ============================================================================

print("="*70)
print("🔴 PRODUCTORES EN ROJO - REQUIEREN ATENCIÓN")
print("="*70)
print()

# Filtrar solo productores en rojo
df_rojos = df_productores[df_productores['SEMAFORO'] == 'ROJO'].copy()

if len(df_rojos) > 0:
    print(f"⚠️  Total de productores en ROJO: {len(df_rojos)}")
    print()
    
    # Ordenar por deuda relativa (mayor a menor)
    df_rojos = df_rojos.sort_values('DEUDA_RELATIVA_%', ascending=False)
    
    # Mostrar top 15 productores en rojo
    print("TOP 15 PRODUCTORES CON MAYOR DEUDA RELATIVA:")
    print("-"*70)
    
    for idx, row in df_rojos.head(15).iterrows():
        print(f"\n🔴 {row['NOMBRE']}")
        print(f"   Código: {row['PROVEEDOR']}")
        print(f"   Deuda: {row['DEUDA_BINS']:,.0f} bins ({row['DEUDA_RELATIVA_%']:.1f}%)")
        print(f"   Bins pedidos: {row['SALIDAS_TOTAL']:,.0f}")
        print(f"   Bins devueltos: {row['ENTRADAS_TOTAL']:,.0f}")
        print(f"   Ratio devolución: {row['RATIO_DEVOLUCION_%']:.1f}%")
        print(f"   Días sin devolver: {row['DIAS_SIN_DEVOLUCION']}")
        print(f"   Razón: {row['RAZON']}")
    
    print()
    print("="*70)
    print("ESTADÍSTICAS DE PRODUCTORES EN ROJO:")
    print("="*70)
    print()
    print(f"   Deuda total:              {df_rojos['DEUDA_BINS'].sum():>15,.0f} bins")
    print(f"   Deuda promedio:           {df_rojos['DEUDA_BINS'].mean():>15,.1f} bins")
    print(f"   Deuda relativa promedio:  {df_rojos['DEUDA_RELATIVA_%'].mean():>15,.1f}%")
    print(f"   Días sin devolver (prom): {df_rojos['DIAS_SIN_DEVOLUCION'].mean():>15,.1f} días")
    print()
    
else:
    print("✅ No hay productores en rojo - Todos están al día")
    print()

print("="*70)
print("✅ CELDA 6 COMPLETADA")
print("="*70)
print()

In [ ]:
# ============================================================================
# CELDA 7: PRODUCTORES EN AMARILLO - REQUIEREN MONITOREO
# ============================================================================

print("="*70)
print("🟡 PRODUCTORES EN AMARILLO - REQUIEREN MONITOREO")
print("="*70)
print()

# Filtrar productores en amarillo
df_amarillos = df_productores[df_productores['SEMAFORO'] == 'AMARILLO'].copy()

if len(df_amarillos) > 0:
    print(f"⚠️  Total de productores en AMARILLO: {len(df_amarillos)}")
    print()
    
    # Ordenar por deuda relativa (mayor a menor)
    df_amarillos = df_amarillos.sort_values('DEUDA_RELATIVA_%', ascending=False)
    
    # Mostrar top 10
    print("TOP 10 PRODUCTORES EN AMARILLO:")
    print("-"*70)
    
    for idx, row in df_amarillos.head(10).iterrows():
        print(f"\n🟡 {row['NOMBRE']}")
        print(f"   Código: {row['PROVEEDOR']}")
        print(f"   Deuda: {row['DEUDA_BINS']:,.0f} bins ({row['DEUDA_RELATIVA_%']:.1f}%)")
        print(f"   Días sin devolver: {row['DIAS_SIN_DEVOLUCION']}")
        print(f"   Razón: {row['RAZON']}")
    
    print()
    print("="*70)
    print("ESTADÍSTICAS DE PRODUCTORES EN AMARILLO:")
    print("="*70)
    print()
    print(f"   Deuda total:              {df_amarillos['DEUDA_BINS'].sum():>15,.0f} bins")
    print(f"   Deuda promedio:           {df_amarillos['DEUDA_BINS'].mean():>15,.1f} bins")
    print(f"   Deuda relativa promedio:  {df_amarillos['DEUDA_RELATIVA_%'].mean():>15,.1f}%")
    print()
    
else:
    print("✅ No hay productores en amarillo")
    print()

print("="*70)
print("✅ CELDA 7 COMPLETADA")
print("="*70)
print()

In [ ]:
# ============================================================================
# CELDA 8: PRODUCTORES EN VERDE - SITUACIÓN ÓPTIMA
# ============================================================================

print("="*70)
print("🟢 PRODUCTORES EN VERDE - SITUACIÓN ÓPTIMA")
print("="*70)
print()

# Filtrar productores en verde
df_verdes = df_productores[df_productores['SEMAFORO'] == 'VERDE'].copy()

if len(df_verdes) > 0:
    print(f"✅ Total de productores en VERDE: {len(df_verdes)}")
    print()
    
    # Estadísticas
    print("ESTADÍSTICAS DE PRODUCTORES EN VERDE:")
    print("-"*70)
    print()
    print(f"   Deuda total:              {df_verdes['DEUDA_BINS'].sum():>15,.0f} bins")
    print(f"   Deuda promedio:           {df_verdes['DEUDA_BINS'].mean():>15,.1f} bins")
    print(f"   Deuda relativa promedio:  {df_verdes['DEUDA_RELATIVA_%'].mean():>15,.1f}%")
    print(f"   Ratio devolución prom:    {df_verdes['RATIO_DEVOLUCION_%'].mean():>15,.1f}%")
    print()
    
    # Productores con mejor comportamiento (mayor ratio de devolución)
    df_verdes_sorted = df_verdes.sort_values('RATIO_DEVOLUCION_%', ascending=False)
    
    print("TOP 5 PRODUCTORES CON MEJOR COMPORTAMIENTO:")
    print("-"*70)
    
    for idx, row in df_verdes_sorted.head(5).iterrows():
        print(f"\n🟢 {row['NOMBRE']}")
        print(f"   Código: {row['PROVEEDOR']}")
        print(f"   Ratio devolución: {row['RATIO_DEVOLUCION_%']:.1f}%")
        print(f"   Deuda: {row['DEUDA_BINS']:,.0f} bins ({row['DEUDA_RELATIVA_%']:.1f}%)")
    
    print()
    
else:
    print("⚠️  No hay productores en verde")
    print()

print("="*70)
print("✅ CELDA 8 COMPLETADA")
print("="*70)
print()

In [ ]:
# ============================================================================
# CELDA 9: TABLA RESUMEN COMPLETA - TODOS LOS PRODUCTORES
# ============================================================================

print("="*70)
print("TABLA RESUMEN - TODOS LOS PRODUCTORES")
print("="*70)
print()

# Ordenar por semáforo (ROJO primero) y luego por deuda relativa
orden_semaforo = {'ROJO': 0, 'AMARILLO': 1, 'VERDE': 2}
df_productores['ORDEN'] = df_productores['SEMAFORO'].map(orden_semaforo)
df_resumen = df_productores.sort_values(['ORDEN', 'DEUDA_RELATIVA_%'], ascending=[True, False])

# Seleccionar columnas para mostrar
columnas_mostrar = [
    'SEMAFORO',
    'PROVEEDOR',
    'NOMBRE',
    'SALIDAS_TOTAL',
    'ENTRADAS_TOTAL',
    'DEUDA_BINS',
    'DEUDA_RELATIVA_%',
    'RATIO_DEVOLUCION_%',
    'DIAS_SIN_DEVOLUCION',
    'RAZON'
]

df_mostrar = df_resumen[columnas_mostrar].copy()

# Renombrar columnas para mejor visualización
df_mostrar.columns = [
    'Semáforo',
    'Código',
    'Nombre',
    'Bins Pedidos',
    'Bins Devueltos',
    'Deuda (bins)',
    'Deuda %',
    'Ratio Dev %',
    'Días Sin Dev',
    'Razón'
]

print(f"Total de productores: {len(df_mostrar):,}")
print()

# Mostrar primeros 30 (prioridad a rojos)
print("PRIMEROS 30 PRODUCTORES (prioridad ROJOS):")
print("="*70)
print()
print(df_mostrar.head(30).to_string(index=False))
print()

# Resumen final
print("="*70)
print("RESUMEN GLOBAL")
print("="*70)
print()

total_bins_pedidos = df_productores['SALIDAS_TOTAL'].sum()
total_bins_devueltos = df_productores['ENTRADAS_TOTAL'].sum()
total_deuda = df_productores['DEUDA_BINS'].sum()
deuda_relativa_global = (total_deuda / total_bins_pedidos * 100) if total_bins_pedidos > 0 else 0

print(f"📊 Bins pedidos totales:        {total_bins_pedidos:>15,.0f}")
print(f"📊 Bins devueltos totales:      {total_bins_devueltos:>15,.0f}")
print(f"📊 Deuda total en campos:       {total_deuda:>15,.0f} bins")
print(f"📊 Deuda relativa global:       {deuda_relativa_global:>15.1f}%")
print()

print(f"🟢 Productores en VERDE:        {conteo.get('VERDE', 0):>15,}")
print(f"🟡 Productores en AMARILLO:     {conteo.get('AMARILLO', 0):>15,}")
print(f"🔴 Productores en ROJO:         {conteo.get('ROJO', 0):>15,}")
print()

print("="*70)
print("CONCORDANCIA CON ANÁLISIS GLOBAL:")
print("="*70)
print()
print(f"✅ La deuda total de {total_deuda:,.0f} bins en campos")
print(f"   debe coincidir con 'BINS EN CAMPOS' del análisis global")
print()
print(f"✅ Si los productores mejoran su ratio de devolución,")
print(f"   aumentará el stock de 'BINS EN PLANTA' automáticamente")
print()

print("="*70)
print("✅ CELDA 9 COMPLETADA")
print("="*70)
print()

In [ ]:
# ============================================================================
# CELDA 10: GRÁFICO - DISTRIBUCIÓN DEL SEMÁFORO
# ============================================================================

import matplotlib.pyplot as plt

print("="*70)
print("GENERANDO GRÁFICOS DE VISUALIZACIÓN")
print("="*70)
print()

# Crear figura con 2 subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7))

# ============================================================================
# GRÁFICO 1: Distribución por semáforo (barras)
# ============================================================================

colores = {
    'VERDE': '#2ECC71',
    'AMARILLO': '#F39C12',
    'ROJO': '#E74C3C'
}

conteo_semaforo = df_productores['SEMAFORO'].value_counts()
colores_grafico = [colores[sem] for sem in conteo_semaforo.index]

ax1.bar(conteo_semaforo.index, conteo_semaforo.values, color=colores_grafico, 
        edgecolor='black', linewidth=2, alpha=0.8)

# Agregar valores sobre las barras
for i, (sem, count) in enumerate(conteo_semaforo.items()):
    porcentaje = (count / len(df_productores)) * 100
    ax1.text(i, count + 2, f'{count}\n({porcentaje:.1f}%)', 
             ha='center', va='bottom', fontsize=14, fontweight='bold')

ax1.set_title('Distribución de Productores por Semáforo', 
              fontsize=16, fontweight='bold', pad=20)
ax1.set_xlabel('Estado del Semáforo', fontsize=13, fontweight='bold')
ax1.set_ylabel('Cantidad de Productores', fontsize=13, fontweight='bold')
ax1.grid(axis='y', alpha=0.3, linestyle='--')
ax1.set_ylim(0, conteo_semaforo.max() * 1.15)

# ============================================================================
# GRÁFICO 2: Deuda total por semáforo
# ============================================================================

deuda_por_semaforo = df_productores.groupby('SEMAFORO')['DEUDA_BINS'].sum()
colores_grafico2 = [colores[sem] for sem in deuda_por_semaforo.index]

ax2.bar(deuda_por_semaforo.index, deuda_por_semaforo.values, color=colores_grafico2,
        edgecolor='black', linewidth=2, alpha=0.8)

# Agregar valores sobre las barras
for i, (sem, deuda) in enumerate(deuda_por_semaforo.items()):
    porcentaje = (deuda / deuda_por_semaforo.sum()) * 100
    ax2.text(i, deuda + 100, f'{deuda:,.0f}\n({porcentaje:.1f}%)', 
             ha='center', va='bottom', fontsize=14, fontweight='bold')

ax2.set_title('Deuda Total por Estado del Semáforo', 
              fontsize=16, fontweight='bold', pad=20)
ax2.set_xlabel('Estado del Semáforo', fontsize=13, fontweight='bold')
ax2.set_ylabel('Deuda Total (bins)', fontsize=13, fontweight='bold')
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{int(x):,}'))
ax2.grid(axis='y', alpha=0.3, linestyle='--')
ax2.set_ylim(0, deuda_por_semaforo.max() * 1.15)

plt.tight_layout()
plt.show()

print("✅ Gráficos generados correctamente")
print()
print("="*70)
print("✅ CELDA 10 COMPLETADA")
print("="*70)
print()

In [ ]:
# ============================================================================
# CELDA 11: EXPORTAR RESULTADOS A EXCEL
# ============================================================================

print("="*70)
print("EXPORTANDO RESULTADOS A EXCEL")
print("="*70)
print()

# Definir ruta de salida
ruta_salida = r"C:\JUGOS\resultados\SEMAFORO_PRODUCTORES.xlsx"

try:
    # Preparar DataFrame para exportar
    df_export = df_resumen[columnas_mostrar].copy()
    df_export.columns = [
        'Semáforo',
        'Código',
        'Nombre',
        'Bins Pedidos',
        'Bins Devueltos',
        'Deuda (bins)',
        'Deuda %',
        'Ratio Devolución %',
        'Días Sin Devolución',
        'Razón'
    ]
    
    # Crear archivo Excel con formato
    with pd.ExcelWriter(ruta_salida, engine='openpyxl') as writer:
        # Hoja 1: Todos los productores
        df_export.to_excel(writer, sheet_name='Todos', index=False)
        
        # Hoja 2: Solo rojos
        if len(df_rojos) > 0:
            df_rojos_export = df_rojos[columnas_mostrar].copy()
            df_rojos_export.columns = df_export.columns
            df_rojos_export.to_excel(writer, sheet_name='Rojos', index=False)
        
        # Hoja 3: Solo amarillos
        if len(df_amarillos) > 0:
            df_amarillos_export = df_amarillos[columnas_mostrar].copy()
            df_amarillos_export.columns = df_export.columns
            df_amarillos_export.to_excel(writer, sheet_name='Amarillos', index=False)
        
        # Hoja 4: Solo verdes
        if len(df_verdes) > 0:
            df_verdes_export = df_verdes[columnas_mostrar].copy()
            df_verdes_export.columns = df_export.columns
            df_verdes_export.to_excel(writer, sheet_name='Verdes', index=False)
        
        # Hoja 5: Resumen
        df_resumen_final = pd.DataFrame({
            'Métrica': [
                'Total Productores',
                'Productores en VERDE',
                'Productores en AMARILLO',
                'Productores en ROJO',
                '',
                'Bins Pedidos Totales',
                'Bins Devueltos Totales',
                'Deuda Total (bins)',
                'Deuda Relativa Global (%)',
                '',
                'Fecha de Análisis',
                'Fecha Inicio Cosecha',
                'Margen de Stockeo (días)',
                'Días Sin Devolución (umbral)',
                'Deuda Relativa Máxima (%)'
            ],
            'Valor': [
                len(df_productores),
                conteo.get('VERDE', 0),
                conteo.get('AMARILLO', 0),
                conteo.get('ROJO', 0),
                '',
                f"{total_bins_pedidos:,.0f}",
                f"{total_bins_devueltos:,.0f}",
                f"{total_deuda:,.0f}",
                f"{deuda_relativa_global:.1f}",
                '',
                FECHA_HOY.strftime('%d/%m/%Y'),
                FECHA_COSECHA.strftime('%d/%m/%Y'),
                MARGEN_STOCKEO,
                DIAS_SIN_DEVOLUCION,
                DEUDA_RELATIVA_MAXIMA
            ]
        })
        df_resumen_final.to_excel(writer, sheet_name='Resumen', index=False)
    
    print(f"✅ Archivo exportado exitosamente:")
    print(f"   {ruta_salida}")
    print()
    print(f"📊 Hojas incluidas:")
    print(f"   - Todos: {len(df_export)} productores")
    if len(df_rojos) > 0:
        print(f"   - Rojos: {len(df_rojos)} productores")
    if len(df_amarillos) > 0:
        print(f"   - Amarillos: {len(df_amarillos)} productores")
    if len(df_verdes) > 0:
        print(f"   - Verdes: {len(df_verdes)} productores")
    print(f"   - Resumen: Estadísticas generales")
    print()
    
except Exception as e:
    print(f"❌ ERROR al exportar: {str(e)}")
    print()

print("="*70)
print("✅ CELDA 11 COMPLETADA")
print("="*70)
print()

In [ ]:
# ============================================================================
# CELDA 12: RESUMEN FINAL Y RECOMENDACIONES
# ============================================================================

print("="*70)
print("                    RESUMEN FINAL - JUGOS S.A.")
print("                 SEMÁFORO POR PRODUCTOR")
print("="*70)
print()

print("📊 RESULTADOS DEL ANÁLISIS:")
print("-"*70)
print()
print(f"   Total productores analizados:     {len(df_productores):>10,}")
print(f"   🟢 Productores en VERDE:          {conteo.get('VERDE', 0):>10,} ({conteo.get('VERDE', 0)/len(df_productores)*100:>5.1f}%)")
print(f"   🟡 Productores en AMARILLO:       {conteo.get('AMARILLO', 0):>10,} ({conteo.get('AMARILLO', 0)/len(df_productores)*100:>5.1f}%)")
print(f"   🔴 Productores en ROJO:           {conteo.get('ROJO', 0):>10,} ({conteo.get('ROJO', 0)/len(df_productores)*100:>5.1f}%)")
print()

print("📈 IMPACTO EN STOCK GLOBAL:")
print("-"*70)
print()
print(f"   Deuda total en campos:            {total_deuda:>10,.0f} bins")
print(f"   Deuda relativa global:            {deuda_relativa_global:>10.1f}%")
print()
print(f"   ✅ CONCORDANCIA: Esta deuda coincide con 'BINS EN CAMPOS'")
print(f"      del análisis global del NOTEBOOK 2")
print()

if len(df_rojos) > 0:
    print("⚠️  ACCIONES RECOMENDADAS:")
    print("-"*70)
    print()
    print(f"   1. Contactar a los {len(df_rojos)} productores en ROJO")
    print(f"   2. Exigir devolución antes de nuevas entregas")
    print(f"   3. Priorizar recuperación de {df_rojos['DEUDA_BINS'].sum():,.0f} bins")
    print(f"   4. Monitorear semanalmente su evolución")
    print()

if len(df_amarillos) > 0:
    print("📋 MONITOREO:")
    print("-"*70)
    print()
    print(f"   - {len(df_amarillos)} productores en AMARILLO requieren seguimiento")
    print(f"   - Revisar su evolución en próximos 7 días")
    print()

print("="*70)
print("✅ ANÁLISIS COMPLETADO EXITOSAMENTE")
print("="*70)
print()
print("📁 Archivos generados:")
print(f"   - {ruta_salida}")
print()
print("🔄 Para actualizar el análisis:")
print("   1. Asegúrate que MOVIMIENTOS.xlsx esté actualizado")
print("   2. Ejecuta todas las celdas nuevamente (Run All)")
print()
print("="*70)